The goal of this notebook is to extract the best individual from each replicate at the end of evolution in a list of EvoTSC runs, in order to first shuffle them and then be able to start a new set of simulations seeded with these individuals.

In [ ]:
import pathlib
import pickle
import re
import numpy as np

In [ ]:
import evotsc_lib
import evotsc_run

In [ ]:
rng = np.random.default_rng(seed=424242)

In [ ]:
main_path = pathlib.Path('/Users/theotime/Desktop/evotsc/phd/param-explor/main/')

In [ ]:
gen = 1_000_000

In [ ]:
def extract_best_indivs(exp_path):

    rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
    params = evotsc_lib.read_params(rep_dirs[0])

    best_indivs_path = exp_path.joinpath(f'best_indivs-{exp_path.name}')
    best_indivs_path.mkdir(exist_ok=True)

    for i_rep, rep_dir in enumerate(rep_dirs):

        best_indiv = evotsc_lib.get_best_indiv(rep_dir, gen)
        
        save_path = best_indivs_path.joinpath(f'best_rep_{i_rep:02}_gen_{gen:07}.evotsc')
        with open(save_path, 'wb') as save_file:
            pickle.dump(best_indiv, save_file)
    
    return best_indivs_path

In [ ]:
main_indivs_path = extract_best_indivs(main_path)

In [ ]:
def create_shuffles(indivs_path, nb_shuffles, rng):
    indiv_paths = sorted([d for d in indivs_path.iterdir() if (d.is_file() and d.name.startswith("best"))])
        
    for indiv_path in indiv_paths:
                
        with open(indiv_path, 'rb') as indiv_file:
            indiv = pickle.load(indiv_file)
                    
        indiv_name = indiv_path.stem
        indiv_dir = indivs_path.joinpath(f'{indiv_name}_shuffled')
        indiv_dir.mkdir(exist_ok=True)
        
        for i_shuffle in range(nb_shuffles):
            shuffled_indiv = evotsc_lib.shuffle_indiv(indiv, indiv.nb_genes, rng)
            shuffled_indiv_path = indiv_dir.joinpath(f'{indiv_name}_shuffled_{i_shuffle:02}.evotsc')
            with open(shuffled_indiv_path, 'wb') as shuffled_indiv_file:
                pickle.dump(shuffled_indiv, shuffled_indiv_file)
        


In [ ]:
create_shuffles(main_indivs_path, 5, rng)